<hr/>

# Data Mining [EN.550.436]
**Tamás Budavári** - budavari@jhu.edu <br/>
**Class 12** - Oct 17, 2016

- Classification exercises
- Cross-validation

<hr/>

<h1><font color="darkblue">Classification</font></h1>

- Based on a **training set** of labeled points, assign class labels to unknown vectors in the **query set**.  

> **Training set**

>$T = \big\{ (x_i, C_i) \big\}$ 

> where $x_i\in \mathbb{R}^d$ are feature sets and $C_i$ are the known class memberships

<nbsp/>

> **Query set**

>$Q = \big\{ x_i \big\}$ where $x_i\in \mathbb{R}^d$ consist of the kind of features in $T$

- And again, $x_i$ are not real vectors but **feature sets** of a bunch of scalars in general

### Bayes with Covariance Matrix

- Estimate the full covariance matrix for the classes

>$\displaystyle {\cal{}L}_{\boldsymbol{x}}(C_k) =  G(\boldsymbol{x};\mu_k, \Sigma_k)$

> Handles correlated features well

- Consider binary problem with 2 classes

> Taking the negative logarithm of the likelihoods we compare

>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma_1^{-1}(x\!-\!\mu_1) + \ln|\Sigma_1|$ vs.

>$\displaystyle (x\!-\!\mu_2)^T\,\Sigma_2^{-1}(x\!-\!\mu_2) + \ln|\Sigma_2|$

> If the difference is lower than a threshold, we classify it accordingly

- This is called **Quadratic Discriminant Analysis**

### Same Covariance Matrix

- When $\Sigma_1=\Sigma_2=\Sigma$, the quadratic terms cancel from the difference
 
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma^{-1}(x\!-\!\mu_1) $ 
>$\displaystyle -\ (x\!-\!\mu_2)^T\,\Sigma^{-1}(x\!-\!\mu_2) $

- Hence this is called **Linear Discriminant Analysis**

> Fewer parameters to estimate during the learning process

> Good, if we don't have enough data, for example...

> Think linear vs quadratic fitting and how you decide between those

### Exercise: QDA 

- Use the provided [training](files/Class12-Train.csv) and [query](files/Class12-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)

> **Query** set only has 2 columns of ($x_i$, $y_i$)

- Place post-it on your laptop to indicate your status and if you need help


In [1]:
%pylab inline
#necessary step

Populating the interactive namespace from numpy and matplotlib


In [2]:
class MyQDA(object):
    """ Template for classifier
    """
    def fit(self,X,C):
        # your code here
        return self

    def predict(self,Y):
        Cpred = None
        # your code here
        # use linalg.det(matrix)
        # and linalg.inv(matrix)
        return Cpred

In [3]:
class MyQDA(dict):
    """ Simple implementation for illustration purposes
    """       
    def fit(self,X,C):#一定要有self
        for k in np.unique(C):   #找不同的类
            members = (C==k)
            prior = members.sum() / float(C.size)#防止得到整数
            S = X[members,:] # subset of class 
            mu = S.mean(axis=0)    #列向量求均值
            Z = (S-mu).T # centered column vectors->row vectors
            cov = Z.dot(Z.T) / (Z[0,:].size-1)  #row.dot(column)
            self[k] = (mu,cov,prior)
        return self
            
    def predict(self,Y):
        Cpred = -1 * ones(Y[:,0].size)
        for i in range(Cpred.size):
            d2min, kbest = 1e99, None
            for k in self:
                mu, cov, prior = self[k]
                diff = (Y[i,:]-mu).T
                d2 = diff.T.dot(linalg.inv(cov)).dot(diff) / 2
                d2 += np.log(linalg.det(cov)) / 2 - np.log(prior) 
                if d2<d2min: d2min,kbest = d2,k
            Cpred[i] = kbest
        return Cpred

In [4]:
# reference implementation
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

Q = np.loadtxt('files/Class12-Query.csv', delimiter=',')
D = np.loadtxt('files/Class12-Train.csv', delimiter=',')
X, C = D[:,0:2], D[:,2]

Cpred = MyQDA().fit(X,C).predict(Q)
Cskit =   QDA().fit(X,C).predict(Q)

print 'Number of different estimates:', (Cpred!=Cskit).sum()

Number of different estimates: 0


<h1><font color="darkblue">Cross-Validation</font></h1>

- How to evaluate the quality of estimator?

> $k$-NN method's parameter affects the results

- We saw on the IRIS data that 1-NN was overfitting

> We discussed excluding the point itself

### Partitions of the Training set

- Random complementary subsets 

> Train on a larger subset, test on a small

> Multiple rounds to decrease variance

### Leave-One-Out

- For each point, we train on the others and test

> Testing on $n$ points requires $n$ training 

- Expensive!

### A Relaxed Variant

- $k$-fold cross-validation 

> 0. Create $k$ partitions of equal sizes, e.g., $k=2$ yields two subsets
> 0. Pick a single partition and train on the other $(k\!-\!1)$ 
> 0. Repeat for all $k$ partitions - requires $k$ trainings

- Leave-One-Out is a special case with $k=n$


### Exercise: Cross-Validation

- Evaluate "QDA" on the [training](files/Class12-Train.csv) set using 2-fold cross-validation

>0. What is the fraction of correct estimates? 
>0. What is the uncertainty of that fraction?
 
> The **training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)


In [10]:
# randomize and split to D1 + D2
np.random.shuffle(D)
split = D[:,0].size / 2
D1, D2 = D[:split,:], D[split:,:]
# train on one estimate on the other
for (i,T,Q) in [(1,D1,D2),(2,D2,D1)]:
    X, C = T[:,0:2], T[:,2]
    Cpred,Ctrue= MyQDA().fit(X,C).predict(Q[:,:2]),Q[:,2]
    print "Case #%d - Number of mislabeled points out of a total %3d points : %2d" \
        % (i, Q.shape[0],(Ctrue!=Cpred).sum()) 

Case #1 - Number of mislabeled points out of a total 157 points : 23
Case #2 - Number of mislabeled points out of a total 156 points : 15


### Done already?

- Visualize the results in the 2D features space
- Make these simple codes run faster 


### Unhomework

- Implement LDA and compare to sklearn

>0. Write code without using sklearn 
>0. Apply to [training](files/Class12-Train.csv) and [query](files/Class12-Query.csv) sets 
>0. Compare your results to sklearn's 

- Perform 10-fold cross-validation of *MyQDA* on [this](files/Class12-Train.csv) file

>0. Write code without using sklearn 
>0. Calculate average and variance of good classifications 
>0. Compare to sklearn 